In [12]:
import os
import csv
import requests
import re
import time
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [2]:
req = requests.get('https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=posglos&NrKadencji=9')
soup = BeautifulSoup(req.text, 'html5lib')

In [22]:
home = 'https://www.sejm.gov.pl/sejm9.nsf/'

session_info = soup.find('tbody').find_all('tr')
all_sessions = []

for row in session_info:
    session_dict = {}
    cells = row.find_all('td')
    
    # Get only info about sessions in 2022
    if '2022' not in cells[1].a.text:
        continue

    session_dict['no'] = cells[0].text.strip()
    session_dict['url'] = os.path.join(home, cells[1].a['href'])
    session_dict['date'] = cells[1].text
    all_sessions.append(session_dict)

# Fill missing session numbers
for session in all_sessions:
    if session['no']:
        current = session['no']
    else:
        session['no'] = current

In [29]:
all_votes = []
vote_dfs = []

for session in all_sessions:# in all_sessions['url']
    
    # Go to the current session url
    req = requests.get(session['url'])
    soup = BeautifulSoup(req.text, 'html5lib')
    vote_info = soup.find('tbody').find_all('tr')

    for i, row in enumerate(vote_info):
        cells = row.find_all('td')

        # This dictionary will serve us later as a mapping for vote topics
        vote_dict = {
            'session_no': session['no'],
            'session_url': session['url'],
            'date': session['date'],
            'vote_no': cells[0].text.strip(),
            'vote_url': os.path.join(home, cells[0].a['href']),
            'vote_time': cells[1].text,
            'vote_topic': cells[2].text,
            'vote_type': cells[2].a.text
        }
        all_votes.append(vote_dict)

        # Go to the current vote url
        time.sleep(1)
        req = requests.get(vote_dict['vote_url'])
        soup = BeautifulSoup(req.text, 'html5lib')
        parties = soup.find_all('td', class_='left')

        # Create empty df for current vote
        cols=['Party', 'MPS', f"{vote_dict['session_no']}/{vote_dict['vote_no']}"]
        if i!=0:
            cols=cols[-1:]
        vote_df = pd.DataFrame(columns=cols)
        
        for party in parties:
            # Get party name
            party_name = party.a.text

            # Go to current party results url
            time.sleep(1)
            req = requests.get(os.path.join(home, party.a['href']))
            soup = BeautifulSoup(req.text, 'html5lib')

            # Get mp names
            mps = soup.find_all('td', class_='left', style='')
            # Get results to current vote df
            vote_results = soup.find_all('td', class_='left', style=re.compile(r'.+'))
            for mp, result in zip(mps, vote_results):
                if i==0:
                    vote_df.loc[len(vote_df)] = party_name, mp.text, result.text
                else:
                    vote_df.loc[len(vote_df)] = result.text

        # Add vote to the list
        vote_dfs.append(vote_df)
    # Get rid of break statement when you ready to get all of it done    
    break

    

In [34]:
pd.concat(vote_dfs, axis=1)

,Party,MPS,62/1,62/2,62/3,62/4,62/5,62/6
0,PiS,Adamczyk Andrzej,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw
1,PiS,Andruszkiewicz Adam,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw
2,PiS,Andzel Waldemar,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw
3,PiS,Ardanowski Jan Krzysztof,Przeciw,Za,Przeciw,Przeciw,Za,Nieobecny
4,PiS,Arent Iwona,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw
...,...,...,...,...,...,...,...,...
455,PS,Ścigaj Agnieszka,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw
456,niez.,Ajchler Zbigniew,Nieobecny,Za,Przeciw,Przeciw,Za,Przeciw
457,niez.,Galla Ryszard,Nieobecny,Nieobecny,Nieobecny,Nieobecny,Nieobecny,Nieobecny
458,niez.,Mejza Łukasz,Przeciw,Za,Przeciw,Przeciw,Za,Przeciw


In [35]:
pd.DataFrame(all_votes)

,session_no,session_url,date,vote_no,vote_url,vote_time,vote_topic,vote_type
0,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,1,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:39:39,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawki nr 1-7 i 9
1,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,2,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:40:10,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawka 8
2,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,3,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:40:33,Pkt. 3 Sprawozdanie Komisji o uchwale Senatu w...,poprawki nr 10-11
3,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,4,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:41:31,Pkt. 2 Pierwsze czytanie poselskiego projektu ...,wniosek o odrzucenie projektu w pierwszym czyt...
4,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,5,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,12:42:25,62. posiedzenie Sejmu Rzeczypospolitej Polskie...,wniosek o zastosowanie art. 39 ust. 3 regulami...
5,62,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,28 września 2022 r.,6,https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?sy...,14:15:41,Pkt. 4 Pierwsze czytanie rządowego projektu us...,rozstrzygnięcie skierowania do komisji.
